In [10]:
import numpy as np
import os
import uproot
import awkward as ak
import ROOT
from cfg.hnl_mva_tools import read_json_file

from data_tools.load_data import (
    get_categorized_data,
    categorize_data,
    read_files_and_open_trees
)
from mva_tools.mva_training_tools import (
    train_one_signal_all_methods,
    load_model,
)

training_vars = ["C_Ds_pt"]
category_list = [1, 2, 3, 4, 5, 6]
category_var = "C_category"

def fix_pred_shape(y_pred, mask):
    """
    This function takes the y_pred 1D array and reshapes it to match the true
    entries of the mask array. False entries are filled with np.nan.
    Args:
        y_pred: the array of predictions
        mask: the array of mask values
    Returns:
        y_pred_shaped: the reshaped array of predictions
    Example:
        y_pred = [1,2,3]
        mask = [[False, True], [True, False, True]]
        y_pred_reshaped = [[nan, 1],[2, nan, 3]]
    """
    assert len(y_pred) == ak.sum(mask)

    y_pred_flat = np.full(ak.sum(ak.num(mask)), np.nan)
    y_pred_flat[ak.flatten(mask)] = y_pred

    y_pred_shaped = ak.unflatten(y_pred_flat, ak.num(mask))
    return y_pred_shaped

def get_bdt_output(data_dict, training_vars, category_list, xgboost_models):
    assert len(category_list) == len(xgboost_models)

    # categorize the data, this adds the C_category column
    categorize_data(
        data_dict,
        category_list,
        category_var=category_var,
        default_category=0,
    )

    # make sure the default category is empty
    assert np.sum(data_dict[category_var] == 0) == 0
    # input_dict[var] is an awkward array
    # make out an awkward array that copies
    # the shape of input_dict[var]
    bdt_output = ak.ones_like(data_dict[training_vars[0]]) * np.nan
    category_arr = data_dict[category_var]

    for category,model in zip(category_list,xgboost_models):
        mask = data_dict[category_var] == category
        # x_cat[i] is ith event
        # x_cat[i][j] is the jth variable for the ith event
        x_cat = np.array([ak.flatten(data_dict[var][mask]) for var in training_vars]).T
        y_score_cat = model.predict(x_cat, output_margin=True)
        y_score_cat_shaped = fix_pred_shape(y_score_cat, mask)

        #ak.where(condition, x, y) does the same as
        # output[i] = x[i] if condition[i] else y[i]     so this fills the 
        #bdt_output array with the predictions for the current category and
        #leaves the rest as they are
        bdt_output = ak.where(mask, y_score_cat_shaped, bdt_output)

    #check that there are no np.nan values left in the bdt_output array
    assert np.sum(np.isnan(bdt_output)) == 0

    #check again that bdt_output has the same shape as the input data
    assert ak.all(ak.num(bdt_output) == ak.num(data_dict[training_vars[0]]))

    return bdt_output, category_arr

KeyboardInterrupt: 

In [2]:
ntuples_json = "cfg/ntuples.json"
vars_json = "cfg/vars_new.json"
my_method = "XGBoost"

# open trees from root files
(
    sig_trees,
    bkg_trees,
    weight_name,
    sig_labels,
    bkg_labels,
) = read_files_and_open_trees(ntuples_json, vars_json)


In [3]:

ntuples = read_json_file(ntuples_json)
signal_file_names = ntuples["signal"]
background_file_names = ntuples["background"]
treename = ntuples["treename"]


good_vars = read_json_file(vars_json)["vars"]
training_vars = read_json_file(vars_json)["training_vars"]

In [4]:
my_sig_tree = sig_trees[2]
my_sig_tree, sig_labels[2]

(<TTree 'final_tree' (314 branches) at 0x7fdd707b3e10>, 'mN1p0_ctau10')

In [5]:
if "C_pass_gen_matching" in good_vars:
    good_vars.remove("C_pass_gen_matching")
data_dict_sig = uproot.concatenate(my_sig_tree,expressions=good_vars, how=dict)
# #remove "C_pass_gen_matching" from the list of variables
# data_dict_bkg = uproot.concatenate(my_bkg_tree,expressions=good_vars,how=dict)

In [6]:
trained_model_dir = "../results_categories/myMVA/mN1p0_ctau10"
#check that dir exists
assert os.path.isdir(trained_model_dir)
xgboost_models = []
for category in category_list:
    category_dir = f"{trained_model_dir}/cat_{category}"
    model = load_model(f"{category_dir}/{my_method}_model", my_method)
    xgboost_models.append(model)

In [7]:
bdt_output_sig, category_sig = get_bdt_output(data_dict_sig, training_vars, category_list, xgboost_models)

In [8]:
bdt_outputs_bkg = []
category_arrs_bkg = []
for bkg_tree in bkg_trees:
    data_dict_bkg = uproot.concatenate(bkg_tree, expressions=good_vars, how=dict)
    bdt_output_bkg, category_arr_bkg = get_bdt_output(data_dict_bkg, training_vars, category_list, xgboost_models)
    bdt_outputs_bkg.append(bdt_output_bkg)
    category_arrs_bkg.append(category_arr_bkg)
    

In [14]:
#check for 0 values at any depth in category_arrs_bkg
for index, category_arr_bkg in enumerate(category_arrs_bkg):
    print(f"bkg tree {index}")
    #category_arr_bkg is an awkward array
    #check if any of the entries are 0
    assert ak.any(category_arr_bkg == 0) == False
    #count total number of entries 
    print("total number of entries")
    print(np.sum(ak.num(category_arr_bkg)))
    #count how many entries are 1,2,3,4,5,6
    print("number of entries in each category, summed")
    print(ak.sum(category_arr_bkg == 1)+ak.sum(category_arr_bkg == 2)+ak.sum(category_arr_bkg == 3)+ak.sum(category_arr_bkg == 4)+ak.sum(category_arr_bkg == 5)+ak.sum(category_arr_bkg == 6))

bkg tree 0
total number of entries
73002
number of entries in each category, summed
73002
bkg tree 1
total number of entries
203404
number of entries in each category, summed
203404
bkg tree 2
total number of entries
290163
number of entries in each category, summed
290163
bkg tree 3
total number of entries
91809
number of entries in each category, summed
91809
bkg tree 4
total number of entries
132291
number of entries in each category, summed
132291
bkg tree 5
total number of entries
333022
number of entries in each category, summed
333022


In [8]:


# def write_score_root_new_file(input_file_name,output_file_name,treename,bdt_score,category_arr):
#     #use uproot to read the tree
#     #just to check consistency

#     with uproot.open(input_file_name) as f:
#         tree = f[treename]
#         #check matching number of entries
#         #and subentries
#         var = list(tree.keys())[0]
#         var_array = tree[var].array(library="ak")
#         assert ak.all(ak.num(var_array) == ak.num(bdt_score))

#     with uproot.recreate(output_file_name) as f:
#         data = {"C_Bdt_score": bdt_score, "C_category": category_arr}
#         f[treename] = ak.zip(data)


In [9]:
# THIS VERSION ADDS A NEW BRANCH CALLED nCand_bdt
# AND FILLS IT WITH THE NUMBER OF CANDIDATES PER EVENT
# AND FILLS THE BDT SCORES IN THE C_Bdt_score BRANCH

def append_score_root(input_file_name,output_file_name,treename,bdt_score):

    #use uproot to read the tree
    #just to check consistency quickly

    max_subevts = 0

    with uproot.open(input_file_name) as f:
        tree = f[treename]
        #check matching number of entries
        #and subentries
        var = list(tree.keys())[0]
        var_array = tree[var].array(library="ak")
        assert ak.all(ak.num(var_array) == ak.num(bdt_score))
        max_subevts = np.nanmax(np.array(ak.num(var_array)))

    

    #now use ROOT TFile to append the score to the tree
    #and save to a new file
        
    # Open the input file
    input_file = ROOT.TFile.Open(input_file_name)
    # Get the tree
    tree = input_file.Get(treename)
    # Create a new file
    output_file = ROOT.TFile.Open(output_file_name, "RECREATE")
    # Clone the tree
    output_tree = tree.CloneTree(0)  # Don't copy the entries yet

    #check total number of events
    assert len(bdt_score) == tree.GetEntries()

    # Create a new branch
    from array import array
    bdt_score_branch = array('d', [0]*max_subevts)
    nCand_bdt = array('i', [0])
    output_tree.Branch('nCand_bdt', nCand_bdt, 'nCand_bdt/I')
    output_tree.Branch('C_Bdt_score', bdt_score_branch, 'C_Bdt_score[nCand_bdt]/D')

    # Now loop over the tree and fill the new branch
    for i in range(tree.GetEntries()):
        tree.GetEntry(i)  # Load the i-th entry
        nCand_bdt[0] = len(bdt_score[i])  # Set the number of candidates
        for j in range(nCand_bdt[0]):
            bdt_score_branch[j] = bdt_score[i][j]  # Set the BDT scores
        output_tree.Fill()  # Fill the new tree

    # Write the new tree to the output file
    output_tree.Write()

    # Close the files
    input_file.Close()
    output_file.Close()

In [48]:
# THIS VERSION USES THE ALREADY EXISTING BRANCH nCand
# BUT IT DOES NOT WORK PERFECTLY BECAUSE nCand IS 
# NOT ALWAYS EQUAL TO THE NUMBER OF CANDIDATES IN THE EVENT
# BUT IT CAN BE LARGER

def append_score_root(input_file_name,output_file_name,treename,bdt_score):
    #use uproot to read the tree
    #just to check consistency

    max_subevts = 0

    with uproot.open(input_file_name) as f:
        tree = f[treename]
        #check matching number of entries
        #and subentries
        var = list(tree.keys())[0]
        var_array = tree[var].array(library="ak")
        assert ak.all(ak.num(var_array) == ak.num(bdt_score))
        max_subevts = np.nanmax(np.array(ak.num(var_array)))

    

    #now use ROOT TFile to append the score to the tree
    #and save to a new file
        
    # Open the input file
    input_file = ROOT.TFile.Open(input_file_name)
    # Get the tree
    tree = input_file.Get(treename)
    # Create a new file
    output_file = ROOT.TFile.Open(output_file_name, "RECREATE")
    # Clone the tree
    output_tree = tree.CloneTree(0)  # Don't copy the entries yet

    #check total number of events
    assert len(bdt_score) == tree.GetEntries()

    # Create a new branch
    from array import array
    bdt_score_branch = array('d', [0]*max_subevts)
    # Get the nCand branch
    nCand_branch = tree.GetBranch('nCand')
    output_tree.Branch('C_Bdt_score', bdt_score_branch, 'C_Bdt_score[nCand]/D')

    # Now loop over the tree and fill the new branch
    for i in range(tree.GetEntries()):
        tree.GetEntry(i)  # Load the i-th entry
        nCand = nCand_branch.GetLeaf('nCand').GetValue()  # Get the value of nCand for this entry
        for j in range(len(bdt_score[i])):
            bdt_score_branch[j] = bdt_score[i][j]  # Set the BDT scores
        output_tree.Fill()  # Fill the new tree

    # Write the new tree to the output file
    output_tree.Write()

    # Close the files
    input_file.Close()
    output_file.Close()

In [10]:
append_score_root(
    input_file_name = signal_file_names[2],
    output_file_name = "test.root",
    treename = treename,
    bdt_score = bdt_output_sig,)

In [ ]:
#compare the two files
with uproot.open(signal_file_names[2]) as f:
    tree = f[treename]
    var = list(tree.keys())[0]
    var_array = tree[var].array(library="ak")
    

In [11]:
#compare nCand of the original file and the new file
nCand_orig = []
with uproot.open(signal_file_names[2]) as f:
    tree = f[treename]
    nCand_orig = tree["nCand"].array(library="np")
with uproot.open("test.root") as f:
    tree = f[treename]
    nCand_new = tree["nCand"].array(library="np")

print(f"total length of nCand_orig: {len(nCand_orig)}, total length of nCand_new: {len(nCand_new)}")
print("element-wise comparison of nCand_orig and nCand_new:")
are_different = False
for i in range(len(nCand_orig)):
    print(f"nCand_orig[{i}]: {nCand_orig[i]}, nCand_new[{i}]: {nCand_new[i]}, index: {i}")
    if nCand_orig[i] != nCand_new[i]:
        #print a large line-skipping space to make it easier to find the difference
        print("\n\n")
        print(f"index {i} is different")
        print("\n\n")
        are_different = True

if are_different:
    print("nCand_orig and nCand_new are different")

    

total length of nCand_orig: 9834, total length of nCand_new: 9834
element-wise comparison of nCand_orig and nCand_new:
nCand_orig[0]: 1, nCand_new[0]: 1, index: 0
nCand_orig[1]: 1, nCand_new[1]: 1, index: 1
nCand_orig[2]: 1, nCand_new[2]: 1, index: 2
nCand_orig[3]: 1, nCand_new[3]: 1, index: 3
nCand_orig[4]: 1, nCand_new[4]: 1, index: 4
nCand_orig[5]: 2, nCand_new[5]: 2, index: 5
nCand_orig[6]: 2, nCand_new[6]: 2, index: 6
nCand_orig[7]: 1, nCand_new[7]: 1, index: 7
nCand_orig[8]: 1, nCand_new[8]: 1, index: 8
nCand_orig[9]: 1, nCand_new[9]: 1, index: 9
nCand_orig[10]: 1, nCand_new[10]: 1, index: 10
nCand_orig[11]: 2, nCand_new[11]: 2, index: 11
nCand_orig[12]: 1, nCand_new[12]: 1, index: 12
nCand_orig[13]: 1, nCand_new[13]: 1, index: 13
nCand_orig[14]: 1, nCand_new[14]: 1, index: 14
nCand_orig[15]: 1, nCand_new[15]: 1, index: 15
nCand_orig[16]: 1, nCand_new[16]: 1, index: 16
nCand_orig[17]: 1, nCand_new[17]: 1, index: 17
nCand_orig[18]: 1, nCand_new[18]: 1, index: 18
nCand_orig[19]: 1,

In [57]:
#open test.root with uproot 
C_Bdt_score = []
orig_var_array = []
with uproot.open("test.root") as f:
    tree = f[treename]
    #check total subentries
    var = list(tree.keys())[0]
    var_array = tree[var].array(library="ak")
    C_Bdt_score = tree["C_Bdt_score"].array(library="ak")
    print(f"total subentries for var: {np.sum(np.array(ak.num(var_array)))}")
    print(f"total subentries for C_Bdt_score: {np.sum(np.array(ak.num(C_Bdt_score)))}")
with uproot.open(signal_file_names[2]) as f:
    tree = f[treename]
    #check total subentries
    var = list(tree.keys())[0]
    var_array = tree[var].array(library="ak")
    orig_var_array = var_array
    print(f"total subentries for var: {np.sum(np.array(ak.num(var_array)))}")

total subentries for var: 11099
total subentries for C_Bdt_score: 11713
total subentries for var: 11099


In [61]:
for index, (a,b) in enumerate(zip(orig_var_array,C_Bdt_score)):
    if len(a) != len(b):
        print(f"index: {index}, len(a): {len(a)}, a:{a}, len(b): {len(b)}, b: {b}") 

index: 95, len(a): 1, a:[1.97], len(b): 2, b: [2.42, -1.74]
index: 121, len(a): 1, a:[1.99], len(b): 3, b: [0.256, -1.74, 1.48]
index: 145, len(a): 1, a:[1.95], len(b): 5, b: [-0.417, -2.74, -2.82, -2.95, 0]
index: 153, len(a): 1, a:[1.95], len(b): 3, b: [2.7, 0.868, -2.82]
index: 162, len(a): 1, a:[2.11], len(b): 2, b: [-1.84, -1.76]
index: 163, len(a): 1, a:[1.97], len(b): 3, b: [-1.58, -1.76, -3.15]
index: 185, len(a): 1, a:[1.97], len(b): 5, b: [1.73, -3.82, -4.6, -2.95, 0]
index: 197, len(a): 1, a:[1.96], len(b): 2, b: [2.09, -2.22]
index: 211, len(a): 2, a:[2.14, 2.3], len(b): 3, b: [-2.39, -2.74, -0.594]
index: 213, len(a): 1, a:[1.96], len(b): 2, b: [1.1, -2.74]
index: 300, len(a): 1, a:[1.97], len(b): 3, b: [-0.296, -0.363, 2.88]
index: 320, len(a): 1, a:[2.22], len(b): 2, b: [-0.741, -0.363]
index: 351, len(a): 1, a:[1.96], len(b): 7, b: [1.14, 2.43, 2.88, -2.95, 0, 0, 0]
index: 370, len(a): 1, a:[1.98], len(b): 2, b: [2.75, -3.89]
index: 376, len(a): 1, a:[1.97], len(b): 5, 

In [ ]:
# pNN_output = np.array([0.5])
# #pNN_output2 = np.array([0.5])
# newBranch1 = mytree.Branch("pNN_output", pNN_output, "pNN_output/D")
# #newBranch2 = mytree.Branch("pNN_output2", pNN_output2, "pNN_output2/D")
# numOfEvents = mytree.GetEntries()

# for n in range(numOfEvents):
#     pNN_output[0] = array_of_pNN[n]
#     #pNN_output2[0] = array_of_pNN2[n]
#     mytree.GetEntry(n)
#     newBranch1.Fill()
#     #newBranch2.Fill()
import array 

def rewrite_root_file(input_file, tree_name, bdt_output): 
    print("ciaone")
    myfile = ROOT.TFile(input_file, "update")
    mytree = myfile.Get(tree_name)
    n_events = mytree.GetEntries()
    print("Save new branch in original ROOT file")

    nCand = array.array('l',[0])
    #set branch address to array
    

    C_testone_2 = array.array('d',(0,)*100)
    new_branch = mytree.Branch("C_testone_2", C_testone_2, "C_testone_2[nCand]/D")
    #print info about the new branch
    for i in range(n_events):
        mytree.SetBranchAddress("nCand", nCand)
        mytree.GetEntry(i)
        print(nCand)
        new_branch.Fill()


    new_branch.Print()
    mytree.Write("", ROOT.TFile.kOverwrite)
    myfile.Close()  


In [ ]:
signal_file_names[2]

'/home/quibus/hnl_ntuples_for_mva/tree_HnlToMuPi_prompt_DsToNMu_NToMuPi_SoftQCDnonD_noQuarkFilter_mN1p0_ctau10p0mm_TuneCP5_13TeV-pythia8-evtgen_tree.root'

In [ ]:
rewrite_root_file(signal_file_names[10], treename, bdt_output_sig)

ciaone
Save new branch in original ROOT file
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [6])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [2])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [2])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [22])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [2])
array('l', [1])
array('l', [2])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [3])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [1])
array('l', [2])
array('l', [1])
array('l', [2])
array('l', [1])
array('l', [1])
array('l',

Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0
Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0
Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0
Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0
Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0
Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0
Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0
Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0
Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0
Error in <TLeafD::GetLen>: Leaf counter is greater than maximum!  leaf: 'C_testone_2' len: 1 max: 0


In [ ]:
signal_file_names[1]

'/home/quibus/hnl_ntuples_for_mva/tree_HnlToMuPi_prompt_DsToNMu_NToMuPi_SoftQCDnonD_noQuarkFilter_mN1p0_ctau100p0mm_TuneCP5_13TeV-pythia8-evtgen_tree.root'

In [ ]:
import uproot 

my_data_dict = {"C_pt": [[1.2, 2.3, 3.4], [4.5, 5.6, 6.7, 7.8]],
                "C_eta": [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6, 0.7]]}
zipped = ak.zip(my_data_dict)
with uproot.recreate("test.root") as f:
    f["mytree"] = zipped


In [ ]:
dummy_bdt_score = ak.Array([[1.1, 2.2, 3.3], [4.4, 5.5, 6.6, 7.7]])
rewrite_root_file("test.root", "mytree", dummy_bdt_score)

Save new branch in original ROOT file
*Br    2 :C_bdtscore_b : C_bdtscore_b[n]/D                                   *
*Entries :        2 : Total  Size=        842 bytes  One basket in memory    *
*Baskets :        0 : Basket Size=      32000 bytes  Compression=   1.00     *
*............................................................................*


In [ ]:
my_root_file = ROOT.TFile("test.root")
my_tree = my_root_file.mytree
for entry in my_tree:
    C_pt = np.array(entry.C_pt)
    C_eta = np.array(entry.C_eta)
    n = np.array(entry.n)
    C_bdtscore_b = np.array(entry.C_bdtscore_b)
    print(f"C_pt: {C_pt}, C_eta: {C_eta}, n: {n}, C_bdtscore_b: {C_bdtscore_b}")

C_pt: [1.2 2.3 3.4], C_eta: [0.1 0.2 0.3], n: 3, C_bdtscore_b: [1.1 2.2 3.3]
C_pt: [4.5 5.6 6.7 7.8], C_eta: [0.4 0.5 0.6 0.7], n: 4, C_bdtscore_b: [4.4 5.5 6.6 7.7]


In [6]:
output_file

'/home/something/scores_asdjio234test$6345.root'